Na camada GOLD, criamos tabelas/visões com:

- Relacionamentos já resolvidos (join entre fato de vendas/pedidos e dimensões como estabelecimentos e produtos, quando existirem);
- Regras de negócio consolidadas (tipos corretos, datas padronizadas, métricas calculadas);
- Baixa complexidade de consulta (consultas diretas e rápidas para o gerente e BI).

**Perguntas de negócio do gerente (e como a GOLD atende)**

**“Quais são as 5 empresas que mais compraram nossos produtos?”**

- identificar os maiores compradores (Top clientes) para foco comercial, renegociação de contratos, programas de fidelização e planejamento de distribuição.
Na GOLD isso é obtido agregando a fato de pedidos/vendas por estabelecimento (empresa) e ordenando por volume (ex.: quantidade) ou valor (faturamento), usando join com a dimensão de estabelecimentos para obter o nome/identificação.

**“Quais são os 5 produtos mais vendidos?”**
 - entender o giro do portfólio para ajustar produção, logística, estoque e campanhas.
Na GOLD isso é calculado agregando a fato por produto (ex.: soma de quantidade vendida) e ordenando para obter o Top 5, com join na dimensão de produtos para descrição/categoria (se disponível).

**“Quais são os 5 produtos que mais geraram faturamento?”**
- priorizar o que mais traz receita e orientar decisões sobre precificação, mix e investimento comercial.
Na GOLD isso é calculado agregando por produto a métrica de faturamento (ex.: quantidade * preco_unitario ou valor_total do pedido), ordenando e retornando o Top 5, novamente enriquecendo via dimensão de produtos.

In [0]:
%sql
-- Criar uma “base analítica” (join entre as tabelas)
-- Como as tabelas são SCD2, na GOLD usaremos apenas os registros vigentes (is_current = true) para responder “o agora”.
-- View base: vw_vendas_atual
-- Join entre fato (pedidos) e dimensão (estabelecimentos), somente versões atuais.

CREATE OR REPLACE VIEW ampev.gold.vw_vendas_atual AS
SELECT
  p.PedidoID,
  p.EstabelecimentoID,
  e.Local      AS estabelecimento,
  e.Email      AS email_estabelecimento,
  e.Telefone   AS telefone_estabelecimento,
  CAST(p.data_venda AS DATE) AS data_venda,
  p.Produto    AS produto,
  p.quantidade_vendida,
  p.Preco_Unitario,
  (p.quantidade_vendida * p.Preco_Unitario) AS valor_total
FROM ampev.silver.fato_pedidos_scd2 p
LEFT JOIN ampev.silver.dim_estabelecimentos_scd2 e
  ON p.EstabelecimentoID = e.EstabelecimentoID
 AND e.is_current = TRUE
WHERE p.is_current = TRUE;


In [0]:
%sql
-- Consultar resultados da tabela view_vendas_atual
SELECT *
FROM ampev.gold.vw_vendas_atual
LIMIT 5;


In [0]:
%sql
 -- Top 5 empresas que mais compraram (por faturamento):

 CREATE OR REPLACE VIEW ampev.gold.top5_empresas_mais_compraram AS
SELECT
  EstabelecimentoID,
  estabelecimento,
  SUM(quantidade_vendida) AS total_unidades,
  SUM(valor_total)        AS total_faturamento
FROM ampev.gold.vw_vendas_atual
GROUP BY EstabelecimentoID, estabelecimento
ORDER BY total_faturamento DESC
LIMIT 5;


In [0]:
%sql
-- Consultar a tabela top_5:

SELECT * FROM ampev.gold.top5_empresas_mais_compraram 

In [0]:
%sql
-- Top 5 produtos mais vendidos (por unidades)

CREATE OR REPLACE VIEW ampev.gold.top5_produtos_mais_vendidos AS
SELECT
  produto,
  SUM(quantidade_vendida) AS total_unidades
FROM ampev.gold.vw_vendas_atual
GROUP BY produto
ORDER BY total_unidades DESC
LIMIT 5;


In [0]:
%sql
-- Consulta da tabela top_produtos_mais_vendidos:

SELECT * FROM ampev.gold.top5_produtos_mais_vendidos

In [0]:
%sql
-- Top 5 produtos que mais geraram faturamento:

CREATE OR REPLACE VIEW ampev.gold.top5_produtos_maior_faturamento AS
SELECT
  produto,
  SUM(valor_total) AS total_faturamento
FROM ampev.gold.vw_vendas_atual
GROUP BY produto
ORDER BY total_faturamento DESC
LIMIT 5;


In [0]:
%sql
-- Consulta da tabela top_produtos_maior_faturamento:

SELECT * FROM ampev.gold.top5_produtos_maior_faturamento

In [0]:
%sql
-- ============================================================
-- ✅ VALIDAÇÃO END-TO-END (SILVER -> GOLD) | AMPEV
-- ============================================================

-- 0) Conferir databases
SHOW DATABASES LIKE 'ampev';

-- 1) Listar objetos Silver
SHOW TABLES IN ampev.silver;

-- 2) Validar existência e schema (Silver)
DESCRIBE TABLE ampev.silver.dim_estabelecimentos_scd2;
DESCRIBE TABLE ampev.silver.fato_pedidos_scd2;

-- 3) Contagens gerais (Silver)
SELECT 'dim_estabelecimentos_scd2' AS tabela, COUNT(*) AS rows FROM ampev.silver.dim_estabelecimentos_scd2
UNION ALL
SELECT 'fato_pedidos_scd2'         AS tabela, COUNT(*) AS rows FROM ampev.silver.fato_pedidos_scd2;

-- 4) Contagens de registros vigentes (SCD2 current)
SELECT 'dim_estabelecimentos_scd2' AS tabela, COUNT(*) AS current_rows
FROM ampev.silver.dim_estabelecimentos_scd2
WHERE is_current = TRUE
UNION ALL
SELECT 'fato_pedidos_scd2'         AS tabela, COUNT(*) AS current_rows
FROM ampev.silver.fato_pedidos_scd2
WHERE is_current = TRUE;

-- 5) Garantir 1 registro vigente por chave (SCD2)
-- Dimensão: 1 current por EstabelecimentoID
SELECT
  EstabelecimentoID,
  COUNT(*) AS qtd_current
FROM ampev.silver.dim_estabelecimentos_scd2
WHERE is_current = TRUE
GROUP BY EstabelecimentoID
HAVING COUNT(*) > 1;

-- Fato: 1 current por chave composta (PedidoID, EstabelecimentoID)
SELECT
  PedidoID,
  EstabelecimentoID,
  COUNT(*) AS qtd_current
FROM ampev.silver.fato_pedidos_scd2
WHERE is_current = TRUE
GROUP BY PedidoID, EstabelecimentoID
HAVING COUNT(*) > 1;

-- 6) Validar datas SCD2 (sanidade)
-- Current deve ter end_date = '9999-12-31' (se você padronizou assim)
SELECT
  'dim_estabelecimentos_scd2' AS tabela,
  COUNT(*) AS current_enddate_errado
FROM ampev.silver.dim_estabelecimentos_scd2
WHERE is_current = TRUE AND end_date <> DATE '9999-12-31'
UNION ALL
SELECT
  'fato_pedidos_scd2' AS tabela,
  COUNT(*) AS current_enddate_errado
FROM ampev.silver.fato_pedidos_scd2
WHERE is_current = TRUE AND end_date <> DATE '9999-12-31';

-- 7) Checar NULLs críticos (Silver)
-- Dim
SELECT
  SUM(CASE WHEN EstabelecimentoID IS NULL THEN 1 ELSE 0 END) AS null_estabelecimentoid,
  SUM(CASE WHEN Local IS NULL THEN 1 ELSE 0 END)            AS null_local
FROM ampev.silver.dim_estabelecimentos_scd2
WHERE is_current = TRUE;

-- Fato
SELECT
  SUM(CASE WHEN PedidoID IS NULL THEN 1 ELSE 0 END)           AS null_pedidoid,
  SUM(CASE WHEN EstabelecimentoID IS NULL THEN 1 ELSE 0 END)  AS null_estabelecimentoid,
  SUM(CASE WHEN data_venda IS NULL THEN 1 ELSE 0 END)         AS null_data_venda,
  SUM(CASE WHEN Produto IS NULL THEN 1 ELSE 0 END)            AS null_produto,
  SUM(CASE WHEN quantidade_vendida IS NULL THEN 1 ELSE 0 END) AS null_quantidade,
  SUM(CASE WHEN Preco_Unitario IS NULL THEN 1 ELSE 0 END)     AS null_preco
FROM ampev.silver.fato_pedidos_scd2
WHERE is_current = TRUE;

-- 8) Validar regras básicas de qualidade (Silver)
SELECT
  COUNT(*) AS qtd_registros_invalidos
FROM ampev.silver.fato_pedidos_scd2
WHERE is_current = TRUE
  AND (quantidade_vendida <= 0 OR Preco_Unitario < 0);

-- 9) Ver objetos Gold
SHOW TABLES IN ampev.gold;
SHOW VIEWS  IN ampev.gold;

-- 10) Schema e amostra da view Gold principal
DESCRIBE TABLE ampev.gold.vw_vendas_atual;

SELECT *
FROM ampev.gold.vw_vendas_atual
LIMIT 20;

-- 11) Join coverage (quantos pedidos ficam sem match de estabelecimento)
SELECT
  COUNT(*) AS total_vendas,
  SUM(CASE WHEN estabelecimento IS NULL THEN 1 ELSE 0 END) AS sem_match_estabelecimento
FROM ampev.gold.vw_vendas_atual;

-- 12) Validar cálculo de valor_total (amostra)
SELECT
  PedidoID,
  EstabelecimentoID,
  Produto,
  quantidade_vendida,
  Preco_Unitario,
  valor_total,
  (quantidade_vendida * Preco_Unitario) AS valor_total_calculado
FROM ampev.gold.vw_vendas_atual
LIMIT 20;

-- 13) Checar Top 5 (resposta ao gerente)
SELECT * FROM ampev.gold.top5_empresas_mais_compraram;
SELECT * FROM ampev.gold.top5_produtos_mais_vendidos;
SELECT * FROM ampev.gold.top5_produtos_maior_faturamento;

-- ============================================================
-- ✅ FIM
-- ============================================================
